In [1]:
import MakingSequences
import makingPairwiseDistancesOurMethod
import makingFinalReconstructedTreeOurMethod
import makingOtherMethodTree

In [2]:
seqLenInput = 1e7
initial_size_Var=100
# time_Var = 200_000
mu_Var = 1e-8
recombination_rate=1e-8

In [3]:
birth_rate = 1000.0
death_rate = 500.0
n_populations = 5
extra_populations = 5 # for "GSA" sampling scheme

In [4]:
seqs, tree = MakingSequences.makeSequencesFromScratch(birth_rate, death_rate, n_populations, extra_populations, seqLenInput, initial_size_Var, mu_Var, recombination_rate)

Demography
╟  Populations
║  ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
║  │ id │name   │description  │initial_size  │ growth_rate │  default_sampling_time│extra_metadata  │
║  ├────────────────────────────────────────────────────────────────────────────────────────────────┤
║  │ 0  │T5     │             │100.0         │      0      │                      0│{}              │
║  │ 1  │T1     │             │100.0         │      0      │                      0│{}              │
║  │ 2  │T2     │             │100.0         │      0      │                      0│{}              │
║  │ 3  │T4     │             │100.0         │      0      │                      0│{}              │
║  │ 4  │T3     │             │100.0         │      0      │                      0│{}              │
║  │ 5  │pop_5  │             │100.0         │      0      │                1.1e+04│{}              │
║  │ 6  │pop_6  │             │100.0         │      0   

In [ ]:
print(tree)
print('====')
pdc = tree.phylogenetic_distance_matrix()
for i, t1 in enumerate(tree.taxon_namespace[:-1]):
    for t2 in tree.taxon_namespace[i+1:]:
        print("Distance between '%s' and '%s': %s" % (t1.label, t2.label, pdc(t1, t2)))
print('====')
tree.print_plot()

In [ ]:
lenSeq = len(seqs['T1'])
mats = []

number_cuts = 10

for i in range(number_cuts):
    cutSeqs = {}
    for key in seqs.keys():
        cutSeqs[key] = seqs[key][int(i * lenSeq/number_cuts) : int((i+1)*lenSeq/number_cuts)]
    ourMatrix = makingPairwiseDistancesOurMethod.makeTimeAndSizeMatrix(cutSeqs, seqLenInput/number_cuts)
    mats.append(ourMatrix)

in getting kmer vectors, working on k: 9 seq 0
in getting kmer vectors, working on k: 9 seq 1
in getting kmer vectors, working on k: 9 seq 2
in getting kmer vectors, working on k: 9 seq 3
in getting kmer vectors, working on k: 9 seq 4
in getting kmer vectors, working on k: 10 seq 0
in getting kmer vectors, working on k: 10 seq 1
in getting kmer vectors, working on k: 10 seq 2
in getting kmer vectors, working on k: 10 seq 3
in getting kmer vectors, working on k: 10 seq 4
in getting kmer vectors, working on k: 11 seq 0
in getting kmer vectors, working on k: 11 seq 1
in getting kmer vectors, working on k: 11 seq 2
in getting kmer vectors, working on k: 11 seq 3
in getting kmer vectors, working on k: 11 seq 4
in getting kmer vectors, working on k: 12 seq 0


In [ ]:
import pandas as pd
import numpy as np

countMat = np.zeros([5,5])

for mat in mats:
    distMat = mat[0]
    for i in range(len(mats)):
        for j in range(len(mats)):
            if distMat[i][j] > -0.05:
                countMat[i][j] += 1
                
print(countMat)

finalDistMat = np.zeros([5,5])

for mat in mats:
    distMat = mat[0]
    for i in range(len(mats)):
        for j in range(len(mats)):
            if distMat[i][j] > 0:
                finalDistMat[i][j] += distMat[i][j] / countMat[i][j]

finalDistMat
df = pd.DataFrame(finalDistMat)
df

In [ ]:
# nj_tree = makingFinalReconstructedTreeOurMethod.makeTree(ourMatrix[0])
# print(nj_tree.as_string("newick"))
# nj_tree.print_plot()

In [ ]:
theirTree = makingOtherMethodTree.inferTreeFromSeqs(seqs, mu_Var)

In [ ]:
theirTree.print_plot()
print(str(theirTree))

In [ ]:
print(str(tree))
tree.print_plot()

In [ ]:
reference_tree = tree
comparison_tree = theirTree

In [ ]:
import collections
import dendropy
import math
import numpy
 


reference_tree.encode_bipartitions()
comparison_tree.encode_bipartitions()
 
ref_edge_lengths = collections.defaultdict(float)
cmp_edge_lengths = collections.defaultdict(float)
 
combined_splits = set()
 
reference_tree_length = 0.0
 
for ref_edge in reference_tree.postorder_edge_iter():
    if ref_edge.head_node != reference_tree.seed_node: # ignore root edge
        ref_edge_lengths[ref_edge.bipartition.split_bitmask] += ref_edge.length
        combined_splits.add(ref_edge.bipartition.split_bitmask)
        reference_tree_length += ref_edge.length

for cmp_edge in comparison_tree.postorder_edge_iter():
    if cmp_edge.head_node != comparison_tree.seed_node: # ignore root edge
        cmp_edge_lengths[cmp_edge.bipartition.split_bitmask] += cmp_edge.length
        combined_splits.add(cmp_edge.bipartition.split_bitmask)

n_combined_splits = len(combined_splits)
 
rf_components = numpy.zeros(n_combined_splits)
weighted_rf_components = numpy.zeros(n_combined_splits)
branch_score_components = numpy.zeros(n_combined_splits)
 
i = 0
 
for split_bitmask in combined_splits:
    if split_bitmask not in cmp_edge_lengths: # only in reference tree
        rf_components[i] += 1.0
        weighted_rf_components[i] += ref_edge_lengths[split_bitmask]
        branch_score_components[i] += ref_edge_lengths[split_bitmask]
    elif split_bitmask not in ref_edge_lengths: # only in comparison tree
        rf_components[i] += 1.0
        weighted_rf_components[i] += cmp_edge_lengths[split_bitmask]
        branch_score_components[i] += cmp_edge_lengths[split_bitmask]
    else: # in both
        branch_score_components[i] += abs(ref_edge_lengths[split_bitmask] - cmp_edge_lengths[split_bitmask])
 
    i += 1

rf_distance = numpy.sum(rf_components)
normalized_rf_distance = rf_distance / float((n_populations - 3) * 2)
weighted_rf_distance = numpy.sum(weighted_rf_components)
euclidean_branch_score = numpy.sqrt(numpy.sum(branch_score_components * branch_score_components))
normalized_branch_score = numpy.sum(branch_score_components) / reference_tree_length
 
# print(reference_tree)
# print(comparison_tree)
 
print(
    rf_distance,
    normalized_rf_distance,
    weighted_rf_distance,
    euclidean_branch_score,
    normalized_branch_score)